In [1]:
import os
import pprint
import numpy as np
import tempfile
import urllib

import absl
import pandas as pd
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx
from tfx.components import CsvExampleGen
from typing import Dict, List, Text
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
#from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
#from tfx.utils.dsl_utils import external_input


%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

In [2]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.4.1
TFX version: 0.30.0


In [3]:
! mkdir tfx
! mkdir tfx/pipelines
! mkdir tfx/metadata
! mkdir tfx/logs
! mkdir tfx/data
! mkdir tfx/serving_model


! mkdir eval_data/

!wget https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv

--2021-06-17 14:06:58--  https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1922812 (1,8M) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]   1,83M  1,60MB/s    in 1,1s    

2021-06-17 14:07:00 (1,60 MB/s) - ‘data.csv’ saved [1922812/1922812]



In [4]:
df = pd.read_csv('data.csv')

##Drop useless columns
df = df.drop(['trip_start_timestamp','trip_miles','pickup_census_tract',
              'dropoff_census_tract','trip_seconds','payment_type','tips', 
              'company','dropoff_community_area','pickup_community_area'], axis=1)

#Drop NA rows
df = df.dropna()

##Keep a test set for final testing( TFX internally splits train and validation data )
np.random.seed(seed=2)
msk = np.random.rand(len(df)) < 0.9
traindf = df[msk]
evaldf = df[~msk]

print(len(traindf))
print(len(evaldf))

traindf.to_csv("tfx/data/data_trans.csv", index=False, header=True)
evaldf.to_csv("eval.csv", index=False, header=False)

13077
1442


In [5]:
##Define all constant
_tfx_root = os.path.join(os.getcwd(), 'tfx');        # Create location ~/tfx
_pipeline_root = os.path.join(_tfx_root, 'pipelines');      # Join ~/tfx/pipelines/
_metadata_db_root = os.path.join(_tfx_root, 'metadata.db');    # Join ~/tfx/metadata.db
_log_root = os.path.join(_tfx_root, 'logs');
_model_root = os.path.join(_tfx_root, 'model');
_data_root = os.path.join(_tfx_root, 'data');
_serving_model_dir = os.path.join(_tfx_root, 'Format-Serving')
_data_filepath = os.path.join(_data_root, "data_trans.csv")

_input_fn_module_file = 'inputfn_trainer.py'
_constants_module_file = 'constants_trainer.py'
_model_trainer_module_file = 'model_trainer.py'

In [6]:
def create_final_pipeline(
    pipeline_name: Text,
    root_path: Text,
    data_path: Text,
    training_params: Dict[Text, Text],
    # beam_pipeline_args: List[Text],
) -> pipeline.Pipeline:

  _pipeline_root = os.path.join(root_path, 'pipelines');     # Join ~/tfx/pipelines/
  _metadata_db_root = os.path.join(root_path, 'metadata.db');    # Join ~/tfx/metadata.db
  _log_root = os.path.join(root_path, 'logs');
  _model_root = os.path.join(root_path, 'model');
  _serving_model_dir = os.path.join(root_path, 'Format-Serving')

  # Full pipeline
  example_gen = CsvExampleGen(input_base=data_path)

  statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])

  infer_schema = SchemaGen(
      statistics=statistics_gen.outputs['statistics'], infer_feature_shape=False)

  validate_stats = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=infer_schema.outputs['schema'])

  trainer = Trainer(
      module_file=os.path.abspath(_model_trainer_module_file),
      custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
      examples=example_gen.outputs['examples'],
      train_args=trainer_pb2.TrainArgs(),
      eval_args=trainer_pb2.EvalArgs(),
      custom_config=(training_params)
      )

  pusher = Pusher(
      model=trainer.outputs['model'],
      push_destination=pusher_pb2.PushDestination(
          filesystem=pusher_pb2.PushDestination.Filesystem(
              base_directory=_serving_model_dir)))

  # This pipeline obj carries the business logic of the pipeline, but no runner-specific information
  # was included.
  return pipeline.Pipeline(
    pipeline_name=  pipeline_name,
    pipeline_root=  root_path,
    components=[
        example_gen, statistics_gen, infer_schema, validate_stats,
        trainer, pusher
    ],
    # metadata_connection_config = metadata.sqlite_metadata_connection_config(_metadata_db_root),
    metadata_connection_config = metadata.sqlite_metadata_connection_config(_metadata_db_root),
    enable_cache=True,
    beam_pipeline_args=['--direct_num_workers=%d' % 0],
  )

In [7]:
#Run pipeline locally
from tfx.orchestration.local.local_dag_runner import LocalDagRunner

##Define all paths
_tfx_root = os.path.join(os.getcwd(), 'tfx')

#Config params
training_params = {"epochs": 50}

#Create and run pipeline
p_ = create_final_pipeline(root_path = _tfx_root, 
                           pipeline_name="local_pipeline", 
                           data_path=_data_root,
                           training_params=training_params)

LocalDagRunner().run(p_)

ERROR:absl:udf_utils.get_fn {'eval_args': '{}', 'custom_config': '{"epochs": 50}', 'module_path': 'model_trainer@/home/limin/my_CICD/tfx/_wheels/tfx_user_code_Trainer-0.0+a8d50ea8986471a27ac7e631700544b1297a7385fa28e725c3812bb59b916c9b-py3-none-any.whl', 'train_args': '{}'} 'run_fn'


2.4.1
INFO:tensorflow:Using MirroredStrategy with devices ('/device:CPU:0',)
INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:CPU:0',), communication = CommunicationImplementation.AUTO
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
pickup_latitude (InputLayer)    [(None, 1)]          0                                            
__________________________________________________________________________________________________
pickup_longitude (InputLayer)   [(None, 1)]          0                                            
__________________________________________________________________________________________________
dropoff_latitude (InputLayer)   [(None, 1)]          0                                            
___________________________________________________________________

__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 16)           0           pickup_latitude[0][0]            
                                                                 pickup_longitude[0][0]           
                                                                 dropoff_latitude[0][0]           
                                                                 dropoff_longitude[0][0]          
                                                                 distance[0][0]                   
                                                                 tf.math.reduce_sum[0][0]         
                                                                 tf.math.reduce_sum_1[0][0]       
                                                                 tf.strings.to_number[0][0]       
                                                                 tf.strings.to_number_1[0][0]     
          

/usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/functional.py:592: UserWarning: Input dict contained keys ['fare'] which did not match any model input. They will be ignored by the model.
  warnings.warn(


100/100 [==============================] - 14s 122ms/step - loss: 218.1682 - rmse: 14.7393 - val_loss: 144.7408 - val_rmse: 12.0303
Epoch 2/50


...

100/100 [==============================] - ETA: 8s - loss: 15.0852 - rmse: 3.883 - ETA: 7s - loss: 14.6681 - rmse: 3.829 - ETA: 8s - loss: 15.2932 - rmse: 3.908 - ETA: 8s - loss: 15.9576 - rmse: 3.990 - ETA: 8s - loss: 142.9245 - rmse: 8.29 - ETA: 7s - loss: 210.3333 - rmse: 10.811 - ETA: 7s - loss: 247.6615 - rmse: 12.369 - ETA: 7s - loss: 268.6482 - rmse: 13.371 - ETA: 7s - loss: 280.0387 - rmse: 14.026 - ETA: 7s - loss: 285.6025 - rmse: 14.455 - ETA: 7s - loss: 287.6481 - rmse: 14.737 - ETA: 7s - loss: 287.2751 - rmse: 14.911 - ETA: 7s - loss: 285.5166 - rmse: 15.015 - ETA: 7s - loss: 282.7094 - rmse: 15.063 - ETA: 7s - loss: 279.2687 - rmse: 15.072 - ETA: 7s - loss: 275.5250 - rmse: 15.056 - ETA: 9s - loss: 271.5104 - rmse: 15.017 - ETA: 10s - loss: 267.3590 - rmse: 14.96 - ETA: 10s - loss: 263.1639 - rmse: 14.89 - ETA: 10s - loss: 258.9513 - rmse: 14.82 - ETA: 9s - loss: 254.7830 - rmse: 14.7380 - ETA: 9s - loss: 250.6809 - rmse: 14.651 - ETA: 9s - loss: 246.6687 - rmse: 14.561 - 

100/100 [==============================] - ETA: 9s - loss: 16.3288 - rmse: 4.040 - ETA: 7s - loss: 17.5275 - rmse: 4.184 - ETA: 7s - loss: 18.1856 - rmse: 4.261 - ETA: 8s - loss: 20.6924 - rmse: 4.524 - ETA: 7s - loss: 21.5862 - rmse: 4.622 - ETA: 7s - loss: 21.7311 - rmse: 4.641 - ETA: 7s - loss: 21.7035 - rmse: 4.641 - ETA: 7s - loss: 21.4606 - rmse: 4.617 - ETA: 7s - loss: 21.1759 - rmse: 4.587 - ETA: 7s - loss: 20.8534 - rmse: 4.552 - ETA: 7s - loss: 20.6512 - rmse: 4.530 - ETA: 7s - loss: 20.4360 - rmse: 4.507 - ETA: 7s - loss: 20.4307 - rmse: 4.507 - ETA: 6s - loss: 20.4419 - rmse: 4.509 - ETA: 6s - loss: 20.4272 - rmse: 4.509 - ETA: 6s - loss: 20.3611 - rmse: 4.502 - ETA: 6s - loss: 20.2669 - rmse: 4.492 - ETA: 6s - loss: 20.1455 - rmse: 4.478 - ETA: 6s - loss: 20.0135 - rmse: 4.464 - ETA: 6s - loss: 19.8966 - rmse: 4.451 - ETA: 6s - loss: 19.7657 - rmse: 4.436 - ETA: 6s - loss: 19.6588 - rmse: 4.424 - ETA: 6s - loss: 19.5373 - rmse: 4.410 - ETA: 6s - loss: 19.4772 - rmse: 4.404

Epoch 47/50
100/100 [==============================] - ETA: 7s - loss: 16.6650 - rmse: 4.082 - ETA: 3s - loss: 18.9511 - rmse: 4.334 - ETA: 4s - loss: 20.3427 - rmse: 4.488 - ETA: 5s - loss: 20.4106 - rmse: 4.500 - ETA: 6s - loss: 20.0857 - rmse: 4.466 - ETA: 6s - loss: 19.6977 - rmse: 4.423 - ETA: 6s - loss: 20.1448 - rmse: 4.473 - ETA: 6s - loss: 20.6314 - rmse: 4.527 - ETA: 6s - loss: 21.0542 - rmse: 4.572 - ETA: 6s - loss: 21.3326 - rmse: 4.603 - ETA: 6s - loss: 21.4553 - rmse: 4.617 - ETA: 6s - loss: 21.5377 - rmse: 4.627 - ETA: 6s - loss: 21.8326 - rmse: 4.659 - ETA: 6s - loss: 22.0380 - rmse: 4.681 - ETA: 6s - loss: 22.3068 - rmse: 4.709 - ETA: 6s - loss: 22.5149 - rmse: 4.731 - ETA: 6s - loss: 22.6625 - rmse: 4.747 - ETA: 6s - loss: 22.7463 - rmse: 4.756 - ETA: 6s - loss: 22.8133 - rmse: 4.764 - ETA: 6s - loss: 22.8550 - rmse: 4.769 - ETA: 6s - loss: 22.8677 - rmse: 4.771 - ETA: 6s - loss: 22.8677 - rmse: 4.771 - ETA: 6s - loss: 22.9690 - rmse: 4.782 - ETA: 6s - loss: 23.0436 -

Epoch 49/50
100/100 [==============================] - ETA: 7s - loss: 10.4171 - rmse: 3.227 - ETA: 8s - loss: 12.4122 - rmse: 3.511 - ETA: 25s - loss: 13.2635 - rmse: 3.63 - ETA: 31s - loss: 13.7554 - rmse: 3.69 - ETA: 27s - loss: 13.9160 - rmse: 3.72 - ETA: 23s - loss: 15.1281 - rmse: 3.86 - ETA: 20s - loss: 16.0398 - rmse: 3.97 - ETA: 18s - loss: 16.5014 - rmse: 4.03 - ETA: 17s - loss: 16.7341 - rmse: 4.06 - ETA: 15s - loss: 16.8000 - rmse: 4.07 - ETA: 14s - loss: 16.9774 - rmse: 4.10 - ETA: 14s - loss: 17.0282 - rmse: 4.10 - ETA: 13s - loss: 17.0386 - rmse: 4.11 - ETA: 12s - loss: 17.0053 - rmse: 4.10 - ETA: 12s - loss: 16.9597 - rmse: 4.10 - ETA: 11s - loss: 16.9151 - rmse: 4.09 - ETA: 11s - loss: 16.8607 - rmse: 4.09 - ETA: 10s - loss: 16.8173 - rmse: 4.08 - ETA: 10s - loss: 16.7591 - rmse: 4.08 - ETA: 9s - loss: 16.6605 - rmse: 4.0708 - ETA: 9s - loss: 16.6510 - rmse: 4.070 - ETA: 9s - loss: 16.6382 - rmse: 4.069 - ETA: 8s - loss: 16.6807 - rmse: 4.074 - ETA: 8s - loss: 16.7960 

INFO:tensorflow:Assets written to: /home/limin/my_CICD/tfx/Trainer/model/3/Format-Serving/assets


2021-06-17 14:17:32.008540: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-06-17 14:17:32.008571: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Traceback (most recent call last):
  File "/usr/local/bin/saved_model_cli", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/tools/saved_model_cli.py", line 1192, in main
    args.func(args)
  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/tools/saved_model_cli.py", line 719, in show
    _show_all(args.dir)
  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/tools/saved_model_cli.py", line 296, in _show_all
    tag_sets = saved_model_utils.get_saved_model_tag_sets(saved_model_dir)
  File "/usr/local/lib/python3.8/dist-pa

In [10]:
#LOCAL: Predict using Keras prediction function
saved_mod = tf.saved_model.load("tfx/Pusher/pushed_model/4")

#Get prediction function from serving
f = saved_mod.signatures['serving_default']

#Run prediction function from serving
f(dropoff_latitude=tf.convert_to_tensor([41.920452]), dropoff_longitude = tf.convert_to_tensor([-87.679955]), pickup_latitude = tf.convert_to_tensor([41.952823]), 
  pickup_longitude =tf.convert_to_tensor([-87.653244]), trip_start_day=tf.convert_to_tensor(["1"]), trip_start_hour=tf.convert_to_tensor(["5"]),
  trip_start_month=tf.convert_to_tensor(["6"]))

{'output_0': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[10.463501]], dtype=float32)>}